# Imports

In [35]:
%matplotlib qt

from IPython import get_ipython
ipython = get_ipython()
ipython.magic("gui qt5") 

import napari
from PIL import Image

import numpy as np
import cv2

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.colors as colors

import time

import pickle as pkl
from os.path import sep

import Cell
import pixels_in_roi
import diff_gauss
import scatter_channels
import hist_channels
import m_dist

# Load images

## Raw images

In [3]:
# %% Load images
t0 = time.time()
folder = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17\\S1_DAPI_546_647_514_594'
plane_nos = range(1, 16)

# base_filename2 = 'S1_dapi_546_488_647_s2z'
base_filename = 'S1_DAPI_546_647_514_594_z'
n = len(plane_nos)
print('Number of planes: {0}'.format(n))

# Create 4D array to store images
img = Image.open('{0}\\{1}{2}c4_ORG.tif'.format(folder, base_filename, str(plane_nos[0]).zfill(2)))

h = img.height
w = img.width
im_array_gad1 = np.zeros([n, h, w])
im_array_vip = np.zeros([n, h, w])
im_array_sst = np.zeros([n, h, w])
im_array_ndnf = np.zeros([n, h, w])
print('Size of image in pixels: {0} X {1} X {2}'.format(n, h, w))

for i in range(n):
    print('Loading image {0}, {1} seconds'.format(i + 1, np.round(time.time() - t0)))
    p = plane_nos[i]
    img_gad1 = Image.open('{0}\{1}{2}c4_ORG.tif'.format(folder, base_filename, str(p).zfill(2)))
    img_vip = Image.open('{0}\{1}{2}c2_ORG.tif'.format(folder, base_filename, str(p).zfill(2)))
    img_sst = Image.open('{0}\{1}{2}c5_ORG.tif'.format(folder, base_filename, str(p).zfill(2)))
    img_ndnf = Image.open('{0}\{1}{2}c3_ORG.tif'.format(folder, base_filename, str(p).zfill(2)))
    try:
        im_array_gad1[i, :, :] = np.array(img_gad1)
        im_array_vip[i, :, :] = np.array(img_vip)
        im_array_sst[i, :, :] = np.array(img_sst)
        im_array_ndnf[i, :, :] = np.array(img_ndnf)
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_gad1 = np.delete(im_array_gad1, i, axis = 0)
        im_array_vip = np.delete(im_array_vip, i, axis = 0)
        im_array_sst = np.delete(im_array_sst, i, axis = 0)
        im_array_ndnf = np.delete(im_array_ndnf, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue

del img_gad1
del img_vip
del img_sst
del img_ndnf

Number of planes: 15
Size of image in pixels: 15 X 2866 X 11160
Loading image 1, 1.0 seconds
Loading image 2, 9.0 seconds
Loading image 3, 19.0 seconds
Loading image 4, 29.0 seconds
Loading image 5, 38.0 seconds
Loading image 6, 48.0 seconds
Loading image 7, 58.0 seconds
Loading image 8, 68.0 seconds
Loading image 9, 78.0 seconds
Loading image 10, 88.0 seconds
Loading image 11, 98.0 seconds
Loading image 12, 107.0 seconds
Loading image 13, 117.0 seconds
Loading image 14, 126.0 seconds
Loading image 15, 136.0 seconds


## Background subtracted (rolling ball)

In [ ]:
# Background subtracted (using rolling ball background subtraction in Fiji)
print('Ndnf')
im_array_ndnf = np.zeros([n, h, w])
img_file = 'S2_Ndnf_bg_sub2'
img = Image.open('{0}\{1}.tif'.format(folder, img_file))
for i in range(n):
    print('Loading image {0}'.format(i + 1))
    p = plane_nos[i]
    try:
        img.seek(p - 1)
        im_array_ndnf[i, :, :] = np.array(img)
        
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_ndnf = np.delete(im_array_ndnf, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue

print('Vip')
im_array_vip = np.zeros([n, h, w])
img_file = 'S2_Vip_bg_sub2'
img = Image.open('{0}\{1}.tif'.format(folder, img_file))
for i in range(n):
    print('Loading image {0}'.format(i + 1))
    p = plane_nos[i]
    try:
        img.seek(p - 1)
        im_array_vip[i, :, :] = np.array(img)
        
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_vip = np.delete(im_array_vip, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue
        
        
print('Sst')
im_array_sst = np.zeros([n, h, w])
img_file = 'S2_Sst_bg_sub2'
img = Image.open('{0}\{1}.tif'.format(folder, img_file))
for i in range(n):
    print('Loading image {0}'.format(i + 1))
    p = plane_nos[i]
    try:
        img.seek(p - 1)
        im_array_sst[i, :, :] = np.array(img)
        
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_sst = np.delete(im_array_sst, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue    

# Render images in napari GUI

## Raw images

In [4]:
viewer = napari.Viewer()
viewer.add_image(im_array_gad1, name = 'Gad1', colormap = 'cyan', blending = 'additive')
viewer.add_image(im_array_ndnf, name = 'Ndnf', colormap = 'magenta', blending = 'additive')
viewer.add_image(im_array_vip, name = 'Vip', colormap = 'yellow', blending = 'additive')
viewer.add_image(im_array_sst, name = 'Sst', colormap = 'green', blending = 'additive')

<Image layer 'Sst' at 0x131f06e4f48>

## Cell masks

In [6]:
mask_layer = viewer.add_shapes(name = 'Cell masks')

indices = list(Cell.cell_data.keys())
for cell in indices:
    if np.mod(cell, 10) == 0:
        print('Cell {0}'.format(cell))
    planes = Cell.cell_data[cell]['z_planes']
    for plane in planes:
        mask = Cell.cell_data[cell]['masks'][plane]
        mask = np.concatenate((np.ones([mask.shape[0], 1])*plane, mask), axis = 1)
        mask_layer.add(mask, shape_type = 'polygon', opacity = 0.2, face_color = 'white', edge_color = 'red', edge_width = 3)

Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280
Cell 290


## Lipofuscin ROIs (annotated)

In [80]:
n_rois = len(l_rois)
print('{0} rois found'.format(n_rois))
viewer.add_shapes(l_rois, name = 'Lipofuscin ROIs',
                   shape_type = 'polygon', opacity = 1, face_color = 'white', 
                   edge_color = 'blue', edge_width = 3)

11 rois found


<Shapes layer 'Lipofuscin ROIs [1]' at 0x1ce6a65db88>

## Filtered images

In [ ]:
viewer.add_image(data = im_diff_ndnf, name = 'Diff ndnf', colormap = 'magenta', blending = 'additive')
viewer.add_image(data = im_diff_sst, name = 'Diff Sst', colormap = 'green', blending = 'additive')
viewer.add_image(data = im_diff_vip, name = 'Diff Vip', colormap = 'yellow', blending = 'additive')

## Binarized images

In [84]:
viewer.add_image(data = im_bin_ndnf, name = 'Binary ndnf', colormap = 'magenta', blending = 'additive')
viewer.add_image(data = im_bin_sst, name = 'Binary Sst', colormap = 'green', blending = 'additive')
viewer.add_image(data = im_bin_vip, name = 'Binary Vip', colormap = 'yellow', blending = 'additive')

<Image layer 'Binary Vip' at 0x1ce6ae6bcc8>

## Detected lipofuscin pixels

In [51]:
viewer.add_image(data = im_array_lipo, name = 'Detected lipofuscin', colormap = 'gray', blending = 'additive')

<Image layer 'Detected lipofuscin' at 0x1380499a088>

## Cell masks after removing lipofuscin (check)

In [52]:
im_array_cells = np.zeros(im_array_gad1.shape)


indices = list(Cell.cell_data.keys())
for cell in indices:
    if np.mod(cell, 10) == 0:
        print('Cell {0}'.format(cell))
    planes = Cell.cell_data[cell]['z_planes']
    for plane in planes:
        mask = cell_pixels_no_lipo[cell][plane]
        mask = np.concatenate((np.ones([1, mask.shape[1]])*plane, mask))
        mask = mask.astype(int)
        im_array_cells[mask[0, :], mask[1, :], mask[2, :]] = np.ones(mask.shape[1])
        

viewer.add_image(data = im_array_cells, name = 'Cell pixels', colormap = 'blue', blending = 'additive')

Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280
Cell 290


<Image layer 'Cell pixels' at 0x138047427c8>

## Cells with significant expression

In [102]:
ndnf_mask_layer = viewer.add_shapes(name = 'Ndnf+ cells')

indices = list(Cell.cell_data.keys())
for cell in indices:
    
    if pv_ndnf[cell - 1] < 0.05:
        planes = Cell.cell_data[cell]['z_planes']
        for plane in planes:
            mask = Cell.cell_data[cell]['masks'][plane]
            mask = np.concatenate((np.ones([mask.shape[0], 1])*plane, mask), axis = 1)
            ndnf_mask_layer.add(mask, shape_type = 'polygon', opacity = 0.2, face_color = 'white', edge_color = 'magenta', edge_width = 3)

In [103]:
vip_mask_layer = viewer.add_shapes(name = 'Vip+ cells')

indices = list(Cell.cell_data.keys())
for cell in indices:
    
    if pv_vip[cell - 1] < 0.05:
        planes = Cell.cell_data[cell]['z_planes']
        for plane in planes:
            mask = Cell.cell_data[cell]['masks'][plane]
            mask = np.concatenate((np.ones([mask.shape[0], 1])*plane, mask), axis = 1)
            vip_mask_layer.add(mask, shape_type = 'polygon', opacity = 0.2, face_color = 'white', edge_color = 'yellow', edge_width = 3)

In [104]:
sst_mask_layer = viewer.add_shapes(name = 'Sst+ cells')

indices = list(Cell.cell_data.keys())
for cell in indices:
    
    if pv_sst[cell - 1] < 0.05:
        planes = Cell.cell_data[cell]['z_planes']
        for plane in planes:
            mask = Cell.cell_data[cell]['masks'][plane]
            mask = np.concatenate((np.ones([mask.shape[0], 1])*plane, mask), axis = 1)
            sst_mask_layer.add(mask, shape_type = 'polygon', opacity = 0.2, face_color = 'white', edge_color = 'green', edge_width = 3)

# Cell segmentation

## Load existing masks

In [5]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file= 'S1_data.pkl'
try:
    with open('{0}\{1}'.format(save_loc, save_file), 'rb') as f:
        Cell.cell_data = pkl.load(f)
        indices = list(Cell.cell_data.keys())
        if not np.max(indices) == len(indices):
            print('Re-numbering cells to be consecutive')
            Cell.cell_data_temp = {}
            for i in range(len(indices)):
                Cell.cell_data_temp[i + 1] = Cell.cell_data[indices[i]]
                Cell.cell_data_temp[i + 1]['cell_id'] = i + 1
            Cell.cell_data = Cell.cell_data_temp 
            with open('{0}\{1}'.format(save_loc, save_file), 'wb') as f:
                pkl.dump(Cell.cell_data, f)
            Cell.n_cells = i + 1
        else:
            Cell.n_cells = len(indices)
    print('{0} cells found'.format(Cell.n_cells))
except:
    print('No data found')

290 cells found


## Get pixels in cell masks

In [8]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
#save_file = 'S2_cell_pixels.pkl'
save_file = 'S1_mask_vertices.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        mask_vertices = pkl.load(f)
        cells = mask_vertices.keys()
        x = 0
        all_cell_pixels = np.zeros([1, 3])
        for cell in cells:
            data = Cell.cell_data[cell]
            planes = data['z_planes']
            for plane in planes:
                xvals = np.reshape(mask_vertices[cell][plane][0], [-1, 1])
                yvals = np.reshape(mask_vertices[cell][plane][1], [-1, 1])
                zvals = np.ones([len(xvals), 1])*plane
                coords = np.concatenate((zvals, xvals, yvals), axis = 1)
                all_cell_pixels = np.concatenate((all_cell_pixels, coords), axis = 0)
        
        all_cell_pixels = all_cell_pixels[1:, :].astype(int)
            
        print('Data loaded')
        
except IOError:
    print('No saved data found, calculating mask pixels')
    xv = range(w)
    yv = range(h)
    coord_array = np.array(np.meshgrid(xv, yv))

    points = np.zeros([h*w, 2])
    p = 0
    for i in range(h):
        for j in range(w):
            points[p, 1] = coord_array[0, i, j]
            points[p, 0] = coord_array[1, i, j]
            p += 1

    no_cells = len(Cell.cell_data.keys())
    mask_vertices = {}
    for cell in range(no_cells):
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        mask_vertices[cell_no] = {}
        cell_dict = Cell.cell_data[cell_no]
        masks = cell_dict['masks']
        z_planes = cell_dict['z_planes']
        for plane in z_planes:

            vertices = masks[plane]
            path = mpltpath.Path(vertices)
            mask = path.contains_points(points)
            mask = np.reshape(mask, [h, w])
            mask_vertices[cell_no][plane] = np.where(mask)
    

Data loaded


## Manual segmentation using Gad1 channel

### Napari keybindings

In [5]:
@viewer.bind_key('n')
def new_cell(viewer):
    global cell1 
    Cell.n_cells += 1
    cell1 = Cell(Cell.n_cells)
    print(Cell.n_cells)

@viewer.bind_key('d')
def get_disc_roi(viewer):
    center = point_layer.data[-1, 1:] # Last drawn point
    seg_layer = viewer.layers[seg_channel]
    z_plane = seg_layer.coordinates[0]
    image = im_array[z_plane, :, :]
    cell_radius_px = 40

    boundary = disc_roi(image, center, cell_radius_px)

    n_theta = boundary.shape[1]
    mask = np.zeros([n_theta, 3])
    mask[:, 0] = np.ones(n_theta)*z_plane
    mask[:, 1:] = np.transpose(boundary)
    mask_layer.add(mask, shape_type= 'polygon')
    
@viewer.bind_key('m')
def add_mask(viewer):
    seg_layer = viewer.layers[seg_channel]
    z_plane = seg_layer.coordinates[0]
    mask = viewer.layers['masks'].data[-1]
    cell1.add_mask(z_plane, mask)
    
@viewer.bind_key('k')
def save_cell(viewer):
    cell1.save_cell()
    
@viewer.bind_key('Control-s')
def save_all(viewer):
    with open('{0}\{1}'.format(save_loc, save_file), 'wb') as f:
                pkl.dump(Cell.cell_data, f)

### One click segmentation

# Lipofuscin exlcusion

## Load existing lipofuscin ROIs

In [16]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file = 'HCR_10.17_S1_lipofuscin_rois_in_cells.pkl'

with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
    l_rois = pkl.load(f)    

## Save lipofuscin rois from napari viewer

In [15]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file = 'HCR_10.17_S1_lipofuscin_rois_in_cells.pkl'

l_rois = viewer.layers['Lipofuscin ROIs'].data
with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
    pkl.dump(l_rois, f) 
    
n_rois = len(l_rois)
print('{0} lipofuscin rois'.format(n_rois))

15 lipofuscin rois


## Get pixels in annotated lipofuscin ROIs

In [17]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file = 'HCR_10.17_S1_lipofuscin_pixels_in_cells.pkl'
    
try:
    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'rb') as f:
        data_dict = pkl.load(f)
        all_lipo_pixels = data_dict['all_lipo_pixels']
        lipo_pixels_roi = data_dict['lipo_pixels_roi']
        print('Data loaded')
        
except IOError:
    print('No saved data found, calculating mask pixels')
    px = pixels_in_roi.pixels_in_roi(h, w, n, l_rois)
    all_lipo_pixels = px['all_pixels']
    lipo_pixels_roi = px['pixels_roi']
    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'wb') as f:
        pkl.dump({'all_lipo_pixels': all_lipo_pixels, 'lipo_pixels_roi': lipo_pixels_roi}, f)

No saved data found, calculating mask pixels
Calculated grid coordinates, 18.0 seconds
ROI 0
ROI 1
ROI 2
ROI 3
ROI 4
ROI 5
ROI 6
ROI 7
ROI 8
ROI 9
ROI 10
ROI 11
ROI 12
ROI 13
ROI 14


## Filter images using difference of gaussians

In [106]:
sigma_small = 2
sigma_large = 5

t0 = time.time()    

img = im_array_ndnf
im_diff_ndnf = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 1)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_sst
im_diff_sst = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 0)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_vip
im_diff_vip = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 0)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

61 seconds
122 seconds
183 seconds


## Binarize filtered images and plot threshold

In [107]:
img_dict = {'Ndnf': im_diff_ndnf, 'Sst': im_diff_sst, 'Vip': im_diff_vip}
#img_dict = {'Ndnf': im_array_ndnf, 'Sst': im_array_sst, 'Vip': im_array_vip}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Cells': all_cell_pixels}

colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}

title = 'Diff gauss sigma = ({0}, {1})'.format(sigma_small, sigma_large)
#title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17\\S1_scatter_plots'
save_file = '{0} hist lipo + cells.png'.format(title)
#save_file = '{0} hist cells.png'.format(title)

thresh_scale = 1

thresh = hist_channels.hist_channels(img_dict, pixels_dict, max_per_group= 100000, 
                                     do_bin = True, bin_group = 'Cells', thresh_scale = thresh_scale,
                                     colors_dict = colors_dict, 
                                     title = title, save = True, save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

im_bin_ndnf = np.zeros(im_diff_ndnf.shape)
im_bin_sst = np.zeros(im_diff_ndnf.shape)
im_bin_vip = np.zeros(im_diff_ndnf.shape)

ix_ndnf = np.where(im_diff_ndnf > thresh['Ndnf'])
ix_sst = np.where(im_diff_sst > thresh['Sst'])
ix_vip = np.where(im_diff_vip > thresh['Vip'])

im_bin_ndnf[ix_ndnf] = im_diff_ndnf[ix_ndnf]
im_bin_sst[ix_sst] = im_diff_sst[ix_sst]
im_bin_vip[ix_vip] = im_diff_vip[ix_vip]

Finding threshold for binarizing
Finding threshold for binarizing
Finding threshold for binarizing


## Find mahalanobis distance of pixels from lipofuscin cloud

In [108]:
img_dict = {'Ndnf': im_bin_ndnf, 'Sst': im_bin_sst, 'Vip': im_bin_vip}
#img_dict = {'Ndnf': im_diff_ndnf, 'Sst': im_diff_sst, 'Vip': im_diff_vip}
#img_dict = {'Ndnf': im_array_ndnf, 'Sst': im_array_sst, 'Vip': im_array_vip}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}

origin_group = 'Lipofuscin'
bin_group = 'Cells'

colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}

title = 'Diff gauss sigma = ({0}, {1}); binarized above {2} std from median'.format(sigma_small, sigma_large, thresh_scale)
#title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17\\S1_scatter_plots'
save_file = '{0} hist lipo + cells.png'.format(title)
#save_file = '{0} hist cells.png'.format(title)

output = m_dist.m_dist(img_dict, pixels_dict, origin_group, bin_group, colors_dict = colors_dict, thresh_scale = 0,
                            title = title, save = True, save_loc = save_loc, save_file = save_file)

m_dist_vals = output['m_dist']
thresh = output['thresh']
cov = output['cov']

del img_dict
del pixels_dict

KeyError: 'cov'

In [109]:
thresh

3.6693125592980986

In [37]:
channel_names = list(img_dict.keys())
n_channels = len(channel_names)

px_groups = list(pixels_dict.keys())


# %% Get values for each channel for each pixel group
px_vals = {}

for group in px_groups:

    px = pixels_dict[group]


    px_vals[group] = {}

    for channel in channel_names:
        px_vals[group][channel] = img_dict[channel][(px[:, 0], px[:, 1], px[:, 2])]


# %% Get mean and covariance matrix of origin group
mu = np.zeros(n_channels)
origin_px = np.zeros([n_channels, px_vals[origin_group][channel_names[0]].shape[0]])
for c in range(n_channels):
    mu[c] = np.mean(px_vals[origin_group][channel_names[c]])
    origin_px[c, :] = px_vals[origin_group][channel_names[c]]

cov = np.cov(origin_px) 
VI = np.linalg.inv(cov)     

In [41]:
plt.figure()
plt.imshow(np.log(cov))
plt.colorbar(label = 'Log covariance')

## Use mahalanobis distance to label all lipofuscin pixels 

In [111]:
lipo_pixels = all_cell_pixels[m_dist_vals['Cells'] < thresh, :]
cell_pixels = all_cell_pixels[m_dist_vals['Cells'] > thresh, :]

im_array_lipo = np.zeros(im_array_ndnf.shape)
im_array_lipo[lipo_pixels[:, 0], lipo_pixels[:, 1], lipo_pixels[:, 2]] = np.ones(lipo_pixels.shape[0])


## Re-calculate cell masks excluding lipofuscin pixels

In [112]:
cell_pixels_no_lipo = {}

cells = mask_vertices.keys()
n_px_total = 0
for cell in cells:
    
    data = Cell.cell_data[cell]
    planes = data['z_planes']
    cell_pixels_no_lipo[cell] = {}
    
    for plane in planes:
        
        cell_pixels_no_lipo[cell][plane] = {}
        n_px = mask_vertices[cell][plane][0].shape[0]
        m = m_dist_vals['Cells'][n_px_total:n_px_total + n_px] # Mahalanobis distance values for pixels in cell mask
        px_coords = np.array(mask_vertices[cell][plane])
        cell_pixels_no_lipo[cell][plane] = px_coords[:, m >= thresh]
        #cell_pixels_no_lipo[cell][plane] = px_coords[:, m ]
        n_px_total += n_px

if not n_px_total == all_cell_pixels.shape[0]:
    print('Total number of pixels incorrect, could have removed incorrect pixels for cell')

## Delete large arrays

In [113]:
del im_diff_ndnf
del im_diff_sst
del im_diff_vip
# del im_bin_ndnf
# del im_bin_vip
# del im_bin_sst

# Classify cells by gene expression

## Calculate intensity for each gene in each cell 

### Average intensity in all pixels in cell mask

In [53]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file = 'S1_data_ndnf_sst_vip.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    avg_intensity_ndnf = sig['Ndnf']
    avg_intensity_sst = sig['Sst'] 
    avg_intensity_vip = sig['Vip']
    print('Data loaded')

except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    avg_intensity_ndnf = np.zeros(no_cells)
    avg_intensity_vip = np.zeros(no_cells)
    avg_intensity_sst = np.zeros(no_cells)

    for cell in range(no_cells):

        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_ndnf = np.zeros(cell_dict['no_planes'])
        intensity_vip = np.zeros(cell_dict['no_planes'])
        intensity_sst = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in z_planes:

            xy_coord = mask_vertices[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            intensity_ndnf[i] = np.mean(im_array_ndnf[plane, x, y])
            intensity_vip[i] = np.mean(im_array_vip[plane, x, y])
            intensity_sst[i] = np.mean(im_array_sst[plane, x, y])
            i += 1

        avg_intensity_ndnf[cell] = np.mean(intensity_ndnf)
        avg_intensity_vip[cell] = np.mean(intensity_vip)
        avg_intensity_sst[cell] = np.mean(intensity_sst)

        with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump({'Ndnf': avg_intensity_ndnf, 'Sst': avg_intensity_sst, 'Vip':avg_intensity_vip}, f)


Data not found, calculating intensities
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280


### Intensity in masks excluding lipofuscin pixels

In [55]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file = 'S1_data_ndnf_sst_vip.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    int_ndnf = sig['Ndnf_lipo_excl']
    int_sst = sig['Sst_lipo_excl'] 
    int_vip = sig['Vip_lipo_excl']
    print('Data loaded')

except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    int_ndnf = np.zeros(no_cells)
    int_vip = np.zeros(no_cells)
    int_sst = np.zeros(no_cells)

    for cell in range(no_cells):

        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_ndnf = np.zeros(cell_dict['no_planes'])
        intensity_vip = np.zeros(cell_dict['no_planes'])
        intensity_sst = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in z_planes:

            xy_coord = cell_pixels_no_lipo[cell_no][plane]
            x = xy_coord[0, :]
            y = xy_coord[1, :]
            intensity_ndnf[i] = np.mean(im_array_ndnf[plane, x, y])
            intensity_vip[i] = np.mean(im_array_vip[plane, x, y])
            intensity_sst[i] = np.mean(im_array_sst[plane, x, y])
            i += 1

        int_ndnf[cell] = np.mean(intensity_ndnf)
        int_vip[cell] = np.mean(intensity_vip)
        int_sst[cell] = np.mean(intensity_sst)
        
        sig['Ndnf_lipo_excl'] = int_ndnf
        sig['Sst_lipo_excl'] = int_sst 
        sig['Vip_lipo_excl'] = int_vip 

        with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump(sig, f)


Data not found, calculating intensities
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280


### Intensity in masks excluding lipofuscin pixels in binarized images (essentially spot counting)

In [114]:
# Note: size of spot is also relevant here. 
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file = 'S1_data_ndnf_sst_vip.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    int_ndnf_bin = sig['Ndnf_lipo_excl_bin']
    int_sst_bin = sig['Sst_lipo_excl_bin'] 
    int_vip_bin = sig['Vip_lipo_excl_bin']
    print('Data loaded')

except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    int_ndnf_bin = np.zeros(no_cells)
    int_vip_bin = np.zeros(no_cells)
    int_sst_bin = np.zeros(no_cells)

    for cell in range(no_cells):

        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_ndnf = np.zeros(cell_dict['no_planes'])
        intensity_vip = np.zeros(cell_dict['no_planes'])
        intensity_sst = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in z_planes:

            xy_coord = cell_pixels_no_lipo[cell_no][plane]
            x = xy_coord[0, :]
            y = xy_coord[1, :]
            intensity_ndnf[i] = np.mean(im_bin_ndnf[plane, x, y])
            intensity_vip[i] = np.mean(im_bin_vip[plane, x, y])
            intensity_sst[i] = np.mean(im_bin_sst[plane, x, y])
            i += 1

        int_ndnf_bin[cell] = np.mean(intensity_ndnf)
        int_vip_bin[cell] = np.mean(intensity_vip)
        int_sst_bin[cell] = np.mean(intensity_sst)
        
    sig['Ndnf_lipo_excl_bin'] = int_ndnf_bin
    sig['Sst_lipo_excl_bin'] = int_sst_bin 
    sig['Vip_lipo_excl_bin'] = int_vip_bin 

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(sig, f)



Data not found, calculating intensities
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280


## Local background

### Raw images

In [92]:

save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file = 'S1_background_ndnf_sst_vip.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        dict = pkl.load(f)
    
    avg_bg_ndnf = dict['Ndnf'] 
    avg_bg_sst = dict['Sst']  
    avg_bg_vip = dict['Vip']
    centers = dict['Centers']
    n_vert_all = dict['n_vert_all']
    bg_all = dict['bg_all']
    
    print('Data loaded')

except:
    
    print('Data not found, calculating background values')
    bg_all = np.ones([n, h, w])
    n_vert_all = {}
    centers = {}

    no_cells = len(Cell.cell_data.keys())
    for cell in range(no_cells):

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']
        n_vert_all[cell] = {}
        centers[cell] = {}

        for plane in z_planes:

            xy_coord = mask_vertices[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            z = plane
            centers[cell][plane] = np.array([np.mean(x), np.mean(y)]).astype(int)


            bg_all[z, x, y] = np.zeros([1, len(x)])

            n_vert_all[cell][plane] = len(x) 
            
    # fig, ax = plt.subplots(nrows = 3, ncols = 20)
    um_per_px = 0.19
    no_cells = len(Cell.cell_data.keys())
    avg_bg_ndnf = np.zeros(no_cells)
    avg_bg_vip = np.zeros(no_cells)
    avg_bg_sst = np.zeros(no_cells)

    local_region = 50 # in um
    local_px = int(local_region/um_per_px/2)

    min_dist = 10 # in um
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_ndnf = np.zeros(len(z_planes))
        bg_vip = np.zeros(len(z_planes))
        bg_sst = np.zeros(len(z_planes))

        p = 0
        for plane in z_planes:

            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)
            local_im_ndnf = im_array_ndnf[plane, x1:x2, y1:y2]
            local_im_sst = im_array_sst[plane, x1:x2, y1:y2]
            local_im_vip = im_array_vip[plane, x1:x2, y1:y2]

    #         ax[0, cell].imshow(local_im, vmin = np.min(local_im), vmax = np.max(local_im)/2)

    #         ax[1, cell].imshow(local_bg)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts = np.min([n_verts, len(order)])

            x_bg = verts[0][order[0:n_verts]]
            y_bg = verts[1][order[0:n_verts]]

            local_bg = np.zeros(local_bg.shape).astype(bool)
            local_bg[x_bg, y_bg] = np.ones(n_verts)

    #         ax[2, cell].imshow(local_bg)

            bg_ndnf[p] = np.mean(local_im_ndnf[local_bg])
            bg_vip[p] = np.mean(local_im_vip[local_bg])
            bg_sst[p] = np.mean(local_im_sst[local_bg])

            p += 1



        avg_bg_ndnf[cell] = np.mean(bg_ndnf)
        avg_bg_vip[cell] = np.mean(bg_vip)
        avg_bg_sst[cell] = np.mean(bg_sst)

        with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump({'Ndnf': avg_bg_ndnf, 'Sst': avg_bg_sst, 'Vip':avg_bg_vip, 'Centers': centers, 
                      'n_vert_all':n_vert_all, 'bg_all': bg_all}, f)
            
        del bg_all

Data loaded


### Binarized images

In [119]:

save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file = 'S1_background_ndnf_sst_vip.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        bg_dict = pkl.load(f)   
    
    centers = bg_dict['Centers']
    n_vert_all = bg_dict['n_vert_all']
    bg_all = bg_dict['bg_all']
    avg_bg_ndnf_bin = bg_dict['Ndnf_bin'] 
    avg_bg_sst_bin = bg_dict['Sst_bin']  
    avg_bg_vip_bin = bg_dict['Vip_bin']

    print('Data loaded')

except:
    
    print('Data not found, calculating background values')
    bg_all = np.ones([n, h, w])
    n_vert_all = {}
    centers = {}

    no_cells = len(Cell.cell_data.keys())
    for cell in range(no_cells):

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']
        n_vert_all[cell] = {}
        centers[cell] = {}

        for plane in z_planes:

            xy_coord = mask_vertices[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            z = plane
            centers[cell][plane] = np.array([np.mean(x), np.mean(y)]).astype(int)


            bg_all[z, x, y] = np.zeros([1, len(x)])

            n_vert_all[cell][plane] = len(x) 
            
    # fig, ax = plt.subplots(nrows = 3, ncols = 20)
    um_per_px = 0.19
    no_cells = len(Cell.cell_data.keys())
    avg_bg_ndnf_bin = np.zeros(no_cells)
    avg_bg_vip_bin = np.zeros(no_cells)
    avg_bg_sst_bin = np.zeros(no_cells)

    local_region = 50 # in um
    local_px = int(local_region/um_per_px/2)

    min_dist = 10 # in um
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):
        cell_no = cell + 1
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_ndnf = np.zeros(len(z_planes))
        bg_vip = np.zeros(len(z_planes))
        bg_sst = np.zeros(len(z_planes))

        p = 0
        for plane in z_planes:

            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)
            local_im_ndnf = im_bin_ndnf[plane, x1:x2, y1:y2]
            local_im_sst = im_bin_sst[plane, x1:x2, y1:y2]
            local_im_vip = im_bin_vip[plane, x1:x2, y1:y2]

    #         ax[0, cell].imshow(local_im, vmin = np.min(local_im), vmax = np.max(local_im)/2)

    #         ax[1, cell].imshow(local_bg)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts = np.min([n_verts, len(order)])

            x_bg = verts[0][order[0:n_verts]]
            y_bg = verts[1][order[0:n_verts]]

            local_bg = np.zeros(local_bg.shape).astype(bool)
            local_bg[x_bg, y_bg] = np.ones(n_verts)

    #         ax[2, cell].imshow(local_bg)

            bg_ndnf[p] = np.mean(local_im_ndnf[local_bg])
            bg_vip[p] = np.mean(local_im_vip[local_bg])
            bg_sst[p] = np.mean(local_im_sst[local_bg])

            p += 1



        avg_bg_ndnf_bin[cell] = np.mean(bg_ndnf)
        avg_bg_vip_bin[cell] = np.mean(bg_vip)
        avg_bg_sst_bin[cell] = np.mean(bg_sst)
        
    bg_dict['Ndnf_bin'] = avg_bg_ndnf_bin 
    bg_dict['Sst_bin'] = avg_bg_sst_bin 
    bg_dict['Vip_bin'] = avg_bg_vip_bin 

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(bg_dict, f)
            
    del bg_all

Data loaded


### Calculate mask centers

In [93]:
bg_all = np.ones([n, h, w])
n_vert_all = {}
centers = {}

no_cells = len(Cell.cell_data.keys())
for cell in range(no_cells):

    cell_no = cell + 1
    cell_dict = Cell.cell_data[cell_no]
    z_planes = cell_dict['z_planes']
    n_vert_all[cell] = {}
    centers[cell] = {}

    for plane in z_planes:

        xy_coord = mask_vertices[cell_no][plane]
        x = xy_coord[0]
        y = xy_coord[1]
        z = plane
        centers[cell][plane] = np.array([np.mean(x), np.mean(y)]).astype(int)


        bg_all[z, x, y] = np.zeros([1, len(x)])

        n_vert_all[cell][plane] = len(x) 

In [78]:
save_file

'S1_background_ndnf_sst_vip.pkl'

In [94]:
with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump({'Ndnf': avg_bg_ndnf, 'Sst': avg_bg_sst, 'Vip':avg_bg_vip, 'Centers': centers,
                     'n_vert_all': n_vert_all, 'bg_all': bg_all}, f)

In [127]:
del all_cell_pixels

## Use background to estimate significance of signal

### Get bootstrap sample means from background

#### Raw images

In [96]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file = 'S1_bootstrap.pkl'

# Parameters
n_sample = 1000 # Number of samples to calculate bootstrap confidence interval
um_per_px = 0.19
local_region = 50 # in um
min_dist = 10 # in um
bg_factor = 4 # Take 4x the number of pixels in the mask of the cell, for local background - it will still be local but you'll
            # get a better estimate of background by averaging over more pixels.

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        d = pkl.load(f)
    bg_bs_ndnf = d['Ndnf']
    bg_bs_sst = d['Sst']
    bg_bs_vip = d['Vip']
    print('Data loaded')
    
except:
    print('No data found, calculating p values')

    no_cells = len(Cell.cell_data.keys())
    bg_bs_ndnf = {}
    bg_bs_sst = {}
    bg_bs_vip = {}

    local_px = int(local_region/um_per_px/2)
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):

        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_bs_ndnf[cell] = {}
        bg_bs_sst[cell] = {}
        bg_bs_vip[cell] = {}
        
        p = 0

        for plane in z_planes:

            bg_bs_ndnf[cell][plane] = np.zeros(n_sample)
            bg_bs_sst[cell][plane] = np.zeros(n_sample)
            bg_bs_vip[cell][plane] = np.zeros(n_sample)
            
            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), 
                                              axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts_bg = np.min([bg_factor*n_verts, len(order)])

            verts = verts[:, order[0:n_verts_bg]]
            samples = np.random.choice(np.linspace(0, n_verts_bg - 1, n_verts_bg).astype(int), 
                                       (n_verts, n_sample), replace = True)
            x_bs = verts[0][samples]
            y_bs = verts[1][samples]

            local_im_ndnf = im_array_ndnf[plane, x1:x2, y1:y2]
            local_im_sst = im_array_sst[plane, x1:x2, y1:y2]
            local_im_vip = im_array_vip[plane, x1:x2, y1:y2]


            local_bg_bs = np.zeros([x2 - x1, y2 - y1]).astype(bool)
            for s in range(n_sample):
                local_bg_bs[x_bs[:, s], y_bs[:, s]] = np.ones(n_verts)
                bg_bs_ndnf[cell][plane][s] = np.mean(local_im_ndnf[local_bg_bs])
                bg_bs_vip[cell][plane][s] = np.mean(local_im_vip[local_bg_bs])
                bg_bs_sst[cell][plane][s] = np.mean(local_im_sst[local_bg_bs])

            p += 1

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump({'Ndnf': bg_bs_ndnf, 'Vip': bg_bs_vip, 'Sst': bg_bs_sst}, f)     
    

No data found, calculating p values
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280


#### Binarized images

In [120]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file = 'S1_bootstrap.pkl'

# Parameters
n_sample = 1000 # Number of samples to calculate bootstrap confidence interval
um_per_px = 0.19
local_region = 50 # in um
min_dist = 10 # in um
bg_factor = 10 # Take 10x the number of pixels in the mask of the cell, for local background - it will still be local but you'll
            # get a better estimate of background by averaging over more pixels.

t0 = time.time()
try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        d = pkl.load(f)
    bg_bs_ndnf_bin = d['Ndnf_bin']
    bg_bs_sst_bin = d['Sst_bin']
    bg_bs_vip_bin = d['Vip_bin']
    print('Data loaded')
    
except:
    print('No data found, calculating p values')

    no_cells = len(Cell.cell_data.keys())
    bg_bs_ndnf_bin = {}
    bg_bs_sst_bin = {}
    bg_bs_vip_bin = {}

    local_px = int(local_region/um_per_px/2)
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):

        if np.mod(cell, 10) == 0:
            t1 = np.round(time.time() - t0)
            print('Cell {0}, {1} seconds'.format(cell, t1))

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_bs_ndnf_bin[cell] = {}
        bg_bs_sst_bin[cell] = {}
        bg_bs_vip_bin[cell] = {}
        
        p = 0

        for plane in z_planes:

            bg_bs_ndnf_bin[cell][plane] = np.zeros(n_sample)
            bg_bs_sst_bin[cell][plane] = np.zeros(n_sample)
            bg_bs_vip_bin[cell][plane] = np.zeros(n_sample)
            
            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), 
                                              axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts_bg = np.min([bg_factor*n_verts, len(order)])

            verts = verts[:, order[0:n_verts_bg]]
            samples = np.random.choice(np.linspace(0, n_verts_bg - 1, n_verts_bg).astype(int), 
                                       (n_verts, n_sample), replace = True)
            x_bs = verts[0][samples]
            y_bs = verts[1][samples]

            local_im_ndnf = im_bin_ndnf[plane, x1:x2, y1:y2]
            local_im_sst = im_bin_sst[plane, x1:x2, y1:y2]
            local_im_vip = im_bin_vip[plane, x1:x2, y1:y2]


            local_bg_bs = np.zeros([x2 - x1, y2 - y1]).astype(bool)
            for s in range(n_sample):
                local_bg_bs[x_bs[:, s], y_bs[:, s]] = np.ones(n_verts)
                bg_bs_ndnf_bin[cell][plane][s] = np.mean(local_im_ndnf[local_bg_bs])
                bg_bs_vip_bin[cell][plane][s] = np.mean(local_im_vip[local_bg_bs])
                bg_bs_sst_bin[cell][plane][s] = np.mean(local_im_sst[local_bg_bs])

            p += 1
            
    d['Ndnf_bin'] = bg_bs_ndnf_bin 
    d['Sst_bin'] = bg_bs_sst_bin 
    d['Vip_bin'] = bg_bs_vip_bin 

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(d, f)     
    

No data found, calculating p values
Cell 0, 1.0 seconds
Cell 10, 6.0 seconds
Cell 20, 12.0 seconds
Cell 30, 18.0 seconds
Cell 40, 23.0 seconds
Cell 50, 30.0 seconds
Cell 60, 36.0 seconds
Cell 70, 43.0 seconds
Cell 80, 50.0 seconds
Cell 90, 57.0 seconds
Cell 100, 63.0 seconds
Cell 110, 69.0 seconds
Cell 120, 75.0 seconds
Cell 130, 82.0 seconds
Cell 140, 86.0 seconds
Cell 150, 92.0 seconds
Cell 160, 97.0 seconds
Cell 170, 102.0 seconds
Cell 180, 106.0 seconds
Cell 190, 111.0 seconds
Cell 200, 116.0 seconds
Cell 210, 121.0 seconds
Cell 220, 129.0 seconds
Cell 230, 134.0 seconds
Cell 240, 138.0 seconds
Cell 250, 143.0 seconds
Cell 260, 151.0 seconds
Cell 270, 157.0 seconds
Cell 280, 162.0 seconds


### Calculate p-value  and confidence interval

In [121]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file1 = 'S1_pvals.pkl'
save_file2 = 'S1_conf_intvls.pkl'
save_file3 = 'S1_pvals_lipo_excl.pkl'
save_file4 = 'S1_pvals_lipo_excl_bin.pkl'

p_value = 0.05 # Threshold for calling signal significantly different from background

try:
    with open('{0}\\{1}'.format(save_loc, save_file1), 'rb') as f:
        p_vals = pkl.load(f)
        pv_ndnf = p_vals['Ndnf']
        pv_sst = p_vals['Sst']
        pv_vip = p_vals['Vip']
    print('Loaded p values for raw images')
    
    with open('{0}\\{1}'.format(save_loc, save_file2), 'rb') as f:
        ci = pkl.load(f)
        ci_ndnf = ci['Ndnf']
        ci_sst = ci['Sst']
        ci_vip = ci['Vip']
    print('Loaded confidence intervals')
    
    with open('{0}\\{1}'.format(save_loc, save_file3), 'rb') as f:
        p_vals = pkl.load(f)
        pv_ndnf2 = p_vals['Ndnf']
        pv_sst2 = p_vals['Sst']
        pv_vip2 = p_vals['Vip']
    print('Loaded p values for lipofuscin excluded masks')
    
    with open('{0}\\{1}'.format(save_loc, save_file4), 'rb') as f:
        p_vals = pkl.load(f)
        pv_ndnf3 = p_vals['Ndnf']
        pv_sst3 = p_vals['Sst']
        pv_vip3 = p_vals['Vip']
    print('Loaded p values for lipofuscin excluded masks in binarized images')


except:
    print('Data not found; calculating')
    
    ci_ndnf = np.zeros(no_cells)
    ci_vip = np.zeros(no_cells)
    ci_sst = np.zeros(no_cells)

    pv_ndnf = np.zeros(no_cells)
    pv_vip = np.zeros(no_cells)
    pv_sst = np.zeros(no_cells)
    
    pv_ndnf2 = np.zeros(no_cells)
    pv_vip2 = np.zeros(no_cells)
    pv_sst2 = np.zeros(no_cells)
    
    pv_ndnf3 = np.zeros(no_cells)
    pv_vip3 = np.zeros(no_cells)
    pv_sst3 = np.zeros(no_cells)
    
    for cell in range(no_cells):

        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bs_means = bg_bs_ndnf[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_ndnf[cell][plane]])
        pv_ndnf[cell] = np.sum(bs_means > avg_intensity_ndnf[cell] - avg_bg_ndnf[cell])/len(bs_means)
        ci_ndnf[cell] = np.mean(np.percentile(bs_means, (1 - p_value)*100, axis = 0))
        pv_ndnf2[cell] = np.sum(bs_means > int_ndnf[cell] - avg_bg_ndnf[cell])/len(bs_means)
        
        bs_means = bg_bs_ndnf_bin[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_ndnf_bin[cell][plane]])
        pv_ndnf3[cell] = np.sum(bs_means >= int_ndnf_bin[cell] - avg_bg_ndnf_bin[cell])/len(bs_means)
        
        bs_means = bg_bs_vip[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_vip[cell][plane]])
        pv_vip[cell] = np.sum(bs_means > avg_intensity_vip[cell] - avg_bg_vip[cell])/len(bs_means)
        pv_vip2[cell] = np.sum(bs_means > int_vip[cell] - avg_bg_vip[cell])/len(bs_means)
        ci_vip[cell] = np.mean(np.percentile(bs_means, (1 - p_value)*100, axis = 0))

        bs_means = bg_bs_vip_bin[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_vip_bin[cell][plane]])
        pv_vip3[cell] = np.sum(bs_means >= int_vip_bin[cell] - avg_bg_vip_bin[cell])/len(bs_means)

        
        bs_means = bg_bs_sst[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_sst[cell][plane]])
        pv_sst[cell] = np.sum(bs_means > avg_intensity_sst[cell] - avg_bg_sst[cell])/len(bs_means)
        pv_sst2[cell] = np.sum(bs_means > int_sst[cell] - avg_bg_sst[cell])/len(bs_means)
        ci_sst[cell] = np.mean(np.percentile(bs_means, (1 - p_value)*100, axis = 0))
        
        bs_means = bg_bs_sst_bin[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_sst_bin[cell][plane]])
        pv_sst3[cell] = np.sum(bs_means >= int_sst_bin[cell] - avg_bg_sst_bin[cell])/len(bs_means)


    with open('{0}\\{1}'.format(save_loc, save_file1), 'wb') as f:
        pkl.dump({'Ndnf': pv_ndnf, 'Sst': pv_sst, 'Vip':pv_vip}, f)

    with open('{0}\\{1}'.format(save_loc, save_file2), 'wb') as f:
        pkl.dump({'Ndnf': ci_ndnf, 'Sst': ci_sst, 'Vip':ci_vip}, f)

    with open('{0}\\{1}'.format(save_loc, save_file3), 'wb') as f:
        pkl.dump({'Ndnf': pv_ndnf2, 'Sst': pv_sst2, 'Vip':pv_vip2}, f)

    with open('{0}\\{1}'.format(save_loc, save_file4), 'wb') as f:
        pkl.dump({'Ndnf': pv_ndnf3, 'Sst': pv_sst3, 'Vip':pv_vip3}, f)

    
print('')
print('With raw images:')

print('Ndnf: {0} cells are significant'.format(np.sum(pv_ndnf < 0.05)))
print('Vip: {0} cells are significant'.format(np.sum(pv_vip < 0.05)))
print('Sst: {0} cells are significant'.format(np.sum(pv_sst < 0.05)))

print('')

print('Ndnf & Sst: {0} cells are significant'.format(np.sum(np.logical_and(pv_ndnf < 0.05, pv_sst < 0.05))))
print('Vip & Sst: {0} cells are significant'.format(np.sum(np.logical_and(pv_vip < 0.05, pv_sst < 0.05))))
print('Ndnf & Vip: {0} cells are significant'.format(np.sum(np.logical_and(pv_ndnf < 0.05, pv_vip < 0.05))))

print('')
print('With lipofuscin pixels exluded:')

print('Ndnf: {0} cells are significant'.format(np.sum(pv_ndnf2 < 0.05)))
print('Vip: {0} cells are significant'.format(np.sum(pv_vip2 < 0.05)))
print('Sst: {0} cells are significant'.format(np.sum(pv_sst2 < 0.05)))

print('')

print('Ndnf & Sst: {0} cells are significant'.format(np.sum(np.logical_and(pv_ndnf2 < 0.05, pv_sst2 < 0.05))))
print('Vip & Sst: {0} cells are significant'.format(np.sum(np.logical_and(pv_vip2 < 0.05, pv_sst2 < 0.05))))
print('Ndnf & Vip: {0} cells are significant'.format(np.sum(np.logical_and(pv_ndnf2 < 0.05, pv_vip2 < 0.05))))

print('')
print('With lipofuscin pixels exluded, in binarized images:')

print('Ndnf: {0} cells are significant'.format(np.sum(pv_ndnf3 < 0.05)))
print('Vip: {0} cells are significant'.format(np.sum(pv_vip3 < 0.05)))
print('Sst: {0} cells are significant'.format(np.sum(pv_sst3 < 0.05)))

print('')

print('Ndnf & Sst: {0} cells are significant'.format(np.sum(np.logical_and(pv_ndnf3 < 0.05, pv_sst3 < 0.05))))
print('Vip & Sst: {0} cells are significant'.format(np.sum(np.logical_and(pv_vip3 < 0.05, pv_sst3 < 0.05))))
print('Ndnf & Vip: {0} cells are significant'.format(np.sum(np.logical_and(pv_ndnf3 < 0.05, pv_vip3 < 0.05))))

Loaded p values for raw images
Loaded confidence intervals
Loaded p values for lipofuscin excluded masks
Data not found; calculating
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280

With raw images:
Ndnf: 78 cells are significant
Vip: 60 cells are significant
Sst: 36 cells are significant

Ndnf & Sst: 17 cells are significant
Vip & Sst: 19 cells are significant
Ndnf & Vip: 7 cells are significant

With lipofuscin pixels exluded:
Ndnf: 68 cells are significant
Vip: 60 cells are significant
Sst: 29 cells are significant

Ndnf & Sst: 14 cells are significant
Vip & Sst: 18 cells are significant
Ndnf & Vip: 7 cells are significant

With lipofuscin pixels exluded, in binarized images:
Ndnf: 107 cells are significant
Vip: 158 cells are significant
Sst: 84 cells are significant

Ndnf & Sst: 62 c

# Find cortex surface and calculate depths

In [98]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17'
save_file = 'S1_depths.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        depths = pkl.load(f)
        print('Depths loaded')
except:
    print('Data not found, draw cortex surface and execute lower cell')

Depths loaded


In [ ]:
um_per_px = 0.19

def find_depth(d1, d2, cell_center): #d1 and d2 are two points on the surface closest to the cell center
    
    dx = d2[0] - d1[0]
    dy = d2[1] - d1[1]
    d1_cell_x = cell_center[0] - d1[0]
    d1_cell_y = cell_center[1] - d1[1]
    t = (dx * d1_cell_x + dy * d1_cell_y) / (dx * dx + dy * dy)
    px = d1[0] + t * dx # Projection of cell center onto line joining d1 and d2
    py = d1[1] + t * dy
    depth = np.linalg.norm([px - cell_center[0], py - cell_center[1]])
    return depth

# Draw surface and calculate depths
surface = viewer.layers['Cell masks'].data[-1]
depths = np.zeros(Cell.n_cells)

for cell in range(Cell.n_cells):
    distances = np.linalg.norm(centers[cell, :] - surface, axis = 1)
    order = np.argsort(distances)
    [p1, p2] = order[0:2]
    d1 = surface[p1]
    d2 = surface[p2]
    depths[cell] = find_depth(d1, d2, centers[cell, :])
depths = depths*um_per_px

# Plots

## 2D scatter plot of pixel values (pairs of channels)

In [26]:
img_dict = {'Ndnf': im_bin_ndnf, 'Sst': im_bin_sst, 'Vip': im_bin_vip}
#img_dict = {'Ndnf': im_diff_ndnf, 'Sst': im_diff_sst, 'Vip': im_diff_vip}
#img_dict = {'Ndnf': im_array_ndnf, 'Sst': im_array_sst, 'Vip': im_array_vip}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Cells': all_cell_pixels}

colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}
title = 'Diff gauss sigma = ({0}, {1}); binarized above {2} std from median'.format(sigma_small, sigma_large, thresh_scale)
#title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17\\S1_scatter_plots'
save_file = '{0} scatter lipo + cells.png'.format(title)
#save_file = '{0} scatter cells.png'.format(title)

scatter_channels.scatter_channels(img_dict, pixels_dict, max_per_group= 100000, colors_dict = colors_dict, 
                                  title = title, save = True, save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

## 3D scatter plot of pixel values (3 channels)

In [44]:
img_dict = {'Ndnf': im_bin_ndnf, 'Sst': im_bin_sst, 'Vip': im_bin_vip}
#img_dict = {'Ndnf': im_diff_ndnf, 'Sst': im_diff_sst, 'Vip': im_diff_vip}
#img_dict = {'Ndnf': im_array_ndnf, 'Sst': im_array_sst, 'Vip': im_array_vip}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Cells': all_cell_pixels}
#pixels_dict = {'Lipofuscin': all_lipo_pixels}


#colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}
#colors_dict = {'Cells': m_dist_vals['Cells'], 'Lipofuscin': m_dist_vals['Lipofuscin']}
colors_dict = {'Cells': np.log(m_dist_vals['Cells']), 'Lipofuscin': 'r'}

title = 'Sub-sampled diff gauss sigma = ({0}, {1}) \n binarized above {2} std from median \n color = log mahalanobis distance from lipofuscin pixels'.format(sigma_small, sigma_large, thresh_scale)
#title = 'Raw images'

save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17\\S1_scatter_plots'
save_file = '3D scatter lipo + cells mahalanobis distance.png'
#save_file = '{0} 3D scatter cells.png'.format(title)
#save_file = '{0} 3D scatter lipo.png'.format(title)
#save_file=  '3D scatter lipo.png'

scatter_channels.scatter_channels(img_dict, pixels_dict, max_per_group= 100000, colors_dict = colors_dict,
                                  make_3D = True,
                                  title = title, save = True, save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

In [47]:
np.max(np.log(m_dist_vals['Cells']))

5.031541064199322

### Simple

In [30]:
ax = plt.axes(projection = '3d')
ax.scatter3D()



## Histogram of pixel values in each channel

In [25]:
#img_dict = {'Ndnf': im_diff_ndnf, 'Sst': im_diff_sst, 'Vip': im_diff_vip}
img_dict = {'Ndnf': im_array_ndnf, 'Sst': im_array_sst, 'Vip': im_array_vip}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Cells': all_cell_pixels}
colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}
#title = 'Diff gauss sigma = ({0}, {1})'.format(sigma_small, sigma_large)
title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17\\S1_scatter_plots'
save_file = '{0} hist lipo + cells.png'.format(title)
#save_file = '{0} hist cells.png'.format(title)

hist_channels.hist_channels(img_dict, pixels_dict, max_per_group= 100000, colors_dict = colors_dict, 
                            title = title, save = True, save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

## Signal vs depth

### Using p-value for significance

In [125]:
# Plot intensity vs depth for all channels
fig, ax = plt.subplots(nrows = 3, ncols = 3)
    
d = depths

npos = pv_ndnf < 0.05
vpos = pv_vip < 0.05
spos = pv_sst < 0.05

n = avg_intensity_ndnf
v = avg_intensity_vip
s = avg_intensity_sst

bn = avg_bg_ndnf
bv = avg_bg_vip
bs = avg_bg_sst

ax[0, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[0, 0].scatter(d[npos], n[npos] - bn[npos], color = 'b', marker = '.')
#ax[0, 0].set_xlabel('Depth from pia (um)')
ax[0, 0].set_ylabel('Intensity in channel (bold => p < 0.05)')
ax[0, 0].set_title('Ndnf: all pixels')

ax[0, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[0, 1].scatter(d[vpos], v[vpos] - bv[vpos], color = 'r', marker = '.')
#ax[0, 1].set_xlabel('Depth from pia (um)')
ax[0, 1].set_title('Vip: all pixels')

ax[0, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[0, 2].scatter(d[spos], s[spos] - bs[spos], color = 'g', marker = '.')
#ax[0, 2].set_xlabel('Depth from pia (um)')
ax[0, 2].set_title('Sst: all pixels')

n = int_ndnf
v = int_vip
s = int_sst

npos2 = pv_ndnf2 < 0.05
vpos2 = pv_vip2 < 0.05
spos2 = pv_sst2 < 0.05

ax[1, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[1, 0].scatter(d[npos2], n[npos2] - bn[npos2], color = 'b', marker = '.')
#ax[1, 0].set_xlabel('Depth from pia (um)')
ax[1, 0].set_ylabel('Intensity in channel (bold => p < 0.05)')
ax[1, 0].set_title('Ndnf: lipofuscin pixels excluded')

ax[1, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[1, 1].scatter(d[vpos2], v[vpos2] - bv[vpos2], color = 'r', marker = '.')
#ax[1, 1].set_xlabel('Depth from pia (um)')
ax[1, 1].set_title('Vip: lipofuscin pixels excluded')

ax[1, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[1, 2].scatter(d[spos2], s[spos2] - bs[spos2], color = 'g', marker = '.')
#ax[1, 2].set_xlabel('Depth from pia (um)')
ax[1, 2].set_title('Sst: lipofuscin pixels excluded')

n = int_ndnf_bin
v = int_vip_bin
s = int_sst_bin

bn = avg_bg_ndnf_bin
bv = avg_bg_vip_bin
bs = avg_bg_sst_bin

npos3 = pv_ndnf3 < 0.05
#vpos3 = pv_vip3 < 0.05
#spos3 = pv_sst3 < 0.05

vpos3 = int_vip_bin - avg_bg_vip_bin > 200
spos3 = int_sst_bin - avg_bg_sst_bin > 500


ax[2, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[2, 0].scatter(d[npos3], n[npos3] - bn[npos3], color = 'b', marker = '.')
ax[2, 0].set_xlabel('Depth from pia (um)')
ax[2, 0].set_ylabel('Intensity in channel (bold => p < 0.05)')
ax[2, 0].set_title('Ndnf: lipofuscin pixels excluded; Binarized images')

ax[2, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[2, 1].scatter(d[vpos3], v[vpos3] - bv[vpos3], color = 'r', marker = '.')
ax[2, 1].set_xlabel('Depth from pia (um)')
ax[2, 1].set_title('Vip: lipofuscin pixels excluded; Binarized images')

ax[2, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[2, 2].scatter(d[spos3], s[spos3] - bs[spos3], color = 'g', marker = '.')
ax[2, 2].set_xlabel('Depth from pia (um)')
ax[2, 2].set_title('Sst: lipofuscin pixels excluded; Binarized images')


Text(0.5, 1.0, 'Sst: lipofuscin pixels excluded; Binarized images')

### Using 95% confidence interval for significance

In [100]:
# Plot intensity vs depth for all channels
fig, ax = plt.subplots(nrows = 2, ncols = 3)
    
d = depths

n = avg_intensity_ndnf
v = avg_intensity_vip
s = avg_intensity_sst

bn = avg_bg_ndnf
bv = avg_bg_vip
bs = avg_bg_sst

npos = n - bn > ci_ndnf
vpos = v - bv > ci_vip
spos = s - bs > ci_sst

ax[0, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[0, 0].scatter(d[npos], n[npos] - bn[npos], color = 'b', marker = '.')
ax[0, 0].set_xlabel('Depth from pia (um)')
ax[0, 0].set_ylabel('Intensity in channel (bold => 95% CI)')
ax[0, 0].set_title('Ndnf: all pixels')

ax[0, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[0, 1].scatter(d[vpos], v[vpos] - bv[vpos], color = 'r', marker = '.')
ax[0, 1].set_xlabel('Depth from pia (um)')
ax[0, 1].set_title('Vip: all pixels')

ax[0, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[0, 2].scatter(d[spos], s[spos] - bs[spos], color = 'g', marker = '.')
ax[0, 2].set_xlabel('Depth from pia (um)')
ax[0, 2].set_title('Sst: all pixels')

n = int_ndnf
v = int_vip
s = int_sst

npos = n - bn > ci_ndnf
vpos = v - bv > ci_vip
spos = s - bs > ci_sst

ax[1, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[1, 0].scatter(d[npos2], n[npos2] - bn[npos2], color = 'b', marker = '.')
ax[1, 0].set_xlabel('Depth from pia (um)')
ax[1, 0].set_ylabel('Intensity in channel (bold => 95% CI)')
ax[1, 0].set_title('Ndnf: lipofuscin pixels excluded')

ax[1, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[1, 1].scatter(d[vpos2], v[vpos2] - bv[vpos2], color = 'r', marker = '.')
ax[1, 1].set_xlabel('Depth from pia (um)')
ax[1, 1].set_title('Vip: lipofuscin pixels excluded')

ax[1, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[1, 2].scatter(d[spos2], s[spos2] - bs[spos2], color = 'g', marker = '.')
ax[1, 2].set_xlabel('Depth from pia (um)')
ax[1, 2].set_title('Sst: lipofuscin pixels excluded')


Text(0.5, 1.0, 'Sst: lipofuscin pixels excluded')